In [ ]:
# Example - https://github.com/cornelliusyudhawijaya/Churn_Causality_Analysis/blob/main/Causal%20Analysis%20Do%20Why.ipynb
    
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

# Prepare Data

In [ ]:
data = pd.read_csv('../../../data/processed.cleveland.data.csv')

## Drop categorical features
## Drop NULL values
data = data.dropna()


## calculate mean of age column
mean = data.loc[:,"age"].mean()
data['age'] = np.where(data['age'] >= mean, 0, 1)

## Make goal column binary
data['Probability'] = np.where(data['Probability'] > 0, 1, 0)

protected_attribute = 'age'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

(dataset_orig.head(5))

In [ ]:
# This dataset size is very small. So repeating it to make it bigger

dataset_orig = pd.concat([dataset_orig]*5, ignore_index=True)

# Default Fairness Metrics

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy
protected_attribute = 'age'

dataset_orig_train,dataset_orig_test = train_test_split(dataset_orig,test_size=0.2,random_state=0)
X_train, y_train =  dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test, y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
clf.fit(X_train,y_train)
# print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'recall'))
# print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'far'))
# print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'precision'))
# print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'accuracy'))
# print("F1 Score :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'F1'))
print("aod :"+protected_attribute,measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'aod'))
print("eod :"+protected_attribute,measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'eod'))
print("SPD:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'SPD'))
print("DI:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'DI'))

# Graph

In [ ]:
# causal_graph = """
# digraph {
# age;
# fbs;
# trestbps;
# chol;
# Probability;
# U[label="Unobserved Confounders"];
# age -> trestbps; trestbps -> Probability;
# chol -> trestbps; chol -> fbs;
# age -> thalach; thalach -> disease;
# age ->fbs; fbs -> disease;
# age -> Probability;
# U->age;U->Probability;
# }
# """

causal_graph = """

digraph {
age;
thalach;
cp;
thal;
sex;
exang;
Probability;
U[label="Unobserved Confounders"];
sex -> Probability;
age -> thalch; thalch -> Probability;
age -> cp; cp -> Probability
age -> exang; exang -> Probability;
thal -> Probability;
age -> Probability;
U->sex;U->age;U->Probability;
}

"""

## dowhy works for binary inputs only
dataset_orig_train['Probability'] = np.where(dataset_orig_train['Probability'] == 0, False, True)
dataset_orig_train[protected_attribute] = np.where(dataset_orig_train[protected_attribute] == 1, True, False)

In [ ]:
# With graph
import time
data1 = dataset_orig_train.copy()
print(data1.shape)
start = time.time()
model=CausalModel(
        data = data1,
        treatment="age",
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))
print(time.time()-start, "=="*20)
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)
print(time.time()-start, "=="*20)
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)
print(time.time()-start, "=="*20)
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)
print(time.time()-start, "=="*20)


In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter", num_strate=1)
print(refutel)